In [5]:
# ## CSE 158/258, Fall 2022: Homework 1
'''
Submitted by: Shivani Bhakta 
This is CSE 258 Homework assignment. 
Credit: 
Some of the code was given as stub and some was inspired/copied from the lecture notes by McAuley, Julian and his book.  
@book{mcauley2022,
      title     = "Personalized Machine Learning",
      author    = "McAuley, Julian",
      year      = "in press",
      publisher = "Cambridge University Press"
    }
'''

'\nSubmitted by: Shivani Bhakta \nThis is CSE 258 Homework assignment. \nCredit: \nSome of the code was given as stub and some was inspired/copied from the lecture notes by McAuley, Julian and his book.  \n@book{mcauley2022,\n      title     = "Personalized Machine Learning",\n      author    = "McAuley, Julian",\n      year      = "in press",\n      publisher = "Cambridge University Press"\n    }\n'

In [6]:
import numpy
import urllib
import scipy.optimize
import random
from sklearn import linear_model
import gzip
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
# from pathlib import Path
# import sys
# sys.path.append("../")
# sys.path.append("../data")

In [7]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [8]:
f = open("data/5year.arff", 'r')

In [9]:
# Read and parse the data
while not '@data' in f.readline():
    pass

dataset = []
for l in f:
    if '?' in l: # Missing entry
        continue
    l = l.split(',')
    values = [1] + [float(x) for x in l]
    values[-1] = values[-1] > 0 # Convert to bool
    dataset.append(values)

In [10]:
X = [d[:-1] for d in dataset]
y = [d[-1] for d in dataset]

In [11]:
answers = {} # Your answers

In [12]:
def accuracy(predictions, y):
    acc = (predictions == y)
    return sum(acc)/len(acc)
    

In [13]:
def BER(predictions, y):
    
    TP = sum([(p and l) for (p,l) in zip(predictions, y)])
    FP = sum([(p and not l) for (p,l) in zip(predictions, y)])
    TN = sum([(not p and not l) for (p,l) in zip(predictions, y)])
    FN = sum([(not p and l) for (p,l) in zip(predictions, y)])
    
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    BER = 1 - 0.5*(TPR + TNR) #balanced Error Rate 
    return BER


In [14]:
### Question 1

In [15]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(X,y)

pred = mod.predict(X)

In [16]:
acc1 = accuracy(pred, y)
ber1 = BER(pred, y)

In [17]:
answers['Q1'] = [acc1, ber1] # Accuracy and balanced error rate

In [18]:
assertFloatList(answers['Q1'], 2)

In [19]:
### Question 2

In [20]:
mod = linear_model.LogisticRegression(C=1, class_weight='balanced')
mod.fit(X,y)

pred = mod.predict(X)

In [21]:
acc2 = accuracy(pred, y)
ber2 = BER(pred, y)

In [22]:
answers['Q2'] = [acc2, ber2]

In [23]:
assertFloatList(answers['Q2'], 2)

In [24]:
### Question 3

In [25]:
random.seed(3)
random.shuffle(dataset)

In [26]:
X = [d[:-1] for d in dataset]
y = [d[-1] for d in dataset]
X[0]

[1,
 0.01897,
 0.57234,
 -0.041892,
 0.8924,
 -261.31,
 0.0,
 0.020566,
 0.7472,
 0.47,
 0.42766,
 0.044181,
 0.052826,
 0.077776,
 0.020566,
 5714.9,
 0.063868,
 1.7472,
 0.020566,
 0.043758,
 144.42,
 1.0607,
 0.019188,
 0.040361,
 0.033148,
 0.37757,
 0.061079,
 0.81253,
 -0.064196,
 5.813,
 1.1652,
 0.084718,
 427.73,
 0.89734,
 0.61039,
 0.13778,
 0.47,
 0.99499,
 0.58993,
 0.29315,
 0.067762,
 0.54236,
 0.040826,
 249.32,
 104.9,
 0.10201,
 0.41473,
 194.3,
 0.0032,
 0.006808,
 0.60702,
 0.38932,
 1.1144,
 0.65533,
 0.904,
 0.8924,
 0.29315,
 0.044357,
 0.7382,
 0.37945,
 2.5274,
 3.4794,
 302.34,
 1.2072,
 0.72022]

In [27]:
Xtrain, Xvalid, Xtest = X[:len(X)//2], X[len(X)//2:(3*len(X))//4], X[(3*len(X))//4:]
ytrain, yvalid, ytest = y[:len(X)//2], y[len(X)//2:(3*len(X))//4], y[(3*len(X))//4:]

In [28]:
len(Xtrain), len(Xvalid), len(Xtest)

(1515, 758, 758)

In [29]:
mod = linear_model.LogisticRegression(C=1, class_weight='balanced')
mod.fit(Xtrain,ytrain)

ypredtrain = mod.predict(Xtrain)
ypredvalid = mod.predict(Xvalid)
ypredTest = mod.predict(Xtest)


In [30]:
berTrain = BER(ypredtrain, ytrain)
berValid = BER(ypredvalid, yvalid)
berTest = BER(ypredTest, ytest)

In [31]:
answers['Q3'] = [berTrain, berValid, berTest]

In [32]:
assertFloatList(answers['Q3'], 3)

In [33]:
### Question 4

In [34]:
## Got most of this code from Chapter 3 workbook and edited it as needed. 

def pipeline(reg):
    mod = linear_model.LogisticRegression(C=reg, class_weight='balanced')
        
    mod.fit(Xtrain,ytrain)
    ypredValid = mod.predict(Xvalid)
    ypredTest = mod.predict(Xtest)
    
    # validation
    
    berVal = BER(ypredValid,yvalid)
    print("C = " + str(reg) + "; validation BER = " + str(berVal))
    
    # test

    berTest = BER(ypredTest,ytest)
#     print("C = " + str(reg) + "; test BER = " + str(berTest))

    return mod, berVal, berTest

modList = []
berList = []
berTestList = []
for c in [1e-04, 1e-03, 1e-02, 1e-01, 1, 10, 100, 1000, 10000]:
    mod, berVal, berTest = pipeline(c)
    modList.append(mod)
    berList.append(berVal) 
    berTestList.append(berTest)

C = 0.0001; validation BER = 0.3281320669380371
C = 0.001; validation BER = 0.31931252826775225
C = 0.01; validation BER = 0.3281320669380371
C = 0.1; validation BER = 0.3179556761646314
C = 1; validation BER = 0.3159203980099503
C = 10; validation BER = 0.3111714156490276
C = 100; validation BER = 0.2955030044582283
C = 1000; validation BER = 0.29618143050978873
C = 10000; validation BER = 0.29618143050978873


In [35]:
answers['Q4'] = berList

In [36]:
assertFloatList(answers['Q4'], 9)

In [37]:
### Question 5

In [38]:
# Since C=100 has the smallest validation error we choose that as a general good performer 
bestC = 100
ber5 = berTestList[6]

In [39]:
answers['Q5'] = [bestC, ber5]

In [40]:
assertFloatList(answers['Q5'], 2)

In [41]:
### Question 6

In [42]:
f = gzip.open("young_adult_10000.json.gz")
dataset = []
for l in f:
    dataset.append(eval(l))

In [43]:
dataTrain = dataset[:9000]
dataTest = dataset[9000:]

In [44]:
len(dataTrain), len(dataTest)

(9000, 1000)

In [45]:
# Some data structures you might want

usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in dataTrain:
    user, item, reviews = d['user_id'], d['book_id'], d['review_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    
    
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)
    ratingDict[(user,item)] = d['rating'] 

In [46]:
def Jaccard(s1, s2):
    num = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    
    if denom == 0:
        return 0
    return num/denom
    

In [47]:
def mostSimilar(i, N):
    similarities = []
    users = usersPerItem[i]
    
    candidateItems = set()
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
        
    for ii in candidateItems: #usersPerItem:
        if ii == i: continue
        sim = Jaccard(users, usersPerItem[ii])
        similarities.append((sim,ii))
    similarities.sort(reverse=True)
    return similarities[:N]
    

In [48]:
answers['Q6'] = mostSimilar('2767052', 10)

In [49]:
assert len(answers['Q6']) == 10
assertFloatList([x[0] for x in answers['Q6']], 10)

In [50]:
### Question 7

In [51]:
userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [52]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        
        if i2 == item: continue
        
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
        
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)\
    
    else:
        # User hasn't rated any similar items
        ratingMean = sum([d['rating'] for d in dataTrain]) / len(dataTrain)
        return ratingMean

In [53]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [54]:
simPredictions = [predictRating(d['user_id'], d['book_id']) for d in dataTest]

In [55]:
labels = [d['rating'] for d in dataTest]
mse7 = MSE(simPredictions, labels)

In [56]:
answers['Q7'] = mse7

In [57]:
assertFloat(answers['Q7'])

In [58]:
### Question 8

In [59]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerItem[item]:
        u2 = d['user_id']
        
        if u2 == user: continue
        
        ratings.append(d['rating'])
        similarities.append(Jaccard(itemsPerUser[item],itemsPerUser[u2]))
        
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    
    else:
        # User hasn't rated any similar items
        ratingMean = sum([d['rating'] for d in dataTrain]) / len(dataTrain)
        return ratingMean

In [60]:
simPredictions = [predictRating(d['user_id'], d['book_id']) for d in dataTest]

In [61]:
labels = [d['rating'] for d in dataTest]
mse8 = MSE(simPredictions, labels)

In [62]:
answers['Q8'] = mse8

In [63]:
assertFloat(answers['Q8'])

In [64]:
f = open("answers_hw2.txt", 'w')
f.write(str(answers) + '\n')
f.close()